In [1]:
from astroquery.vizier import Vizier
import numpy as np
import pyds9
from glob import glob
from astropy.io import fits
from pyds9 import *
from astropy import wcs
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from regions import read_ds9, write_ds9
from regions import DS9Parser
from regions import ds9_objects_to_string
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion
import pandas as pd


# This notebook will detect sources from a vizier catalogue  in a give file(s) using ds9.

<b> This notebook must be run off the NRAO server <b>

In [2]:
def pipeline(files,catalog='VIII/100/gleamegc'):
    Vizier.ROW_LIMIT = -1
    table = Vizier.get_catalogs(catalog='VIII/100/gleamegc')[0]
    Gleam_ra = table['RAJ2000']
    Gleam_dec = table['DEJ2000']
    Gleam_flux = table['Fintwide']
    
    #define a ra/dec mask so only sources in mosaics are chosen
    #I should edit this, seems like cuts arent very good...
    deccuts , racuts, fluxmin = [-41,-21] , [0,161] , 3 #deg,deg,Jy
    #create a mask for ra,dec and flux
    w = []
    for i in range(len(Gleam_dec)):
        w.append((deccuts[0] < Gleam_dec[i] < deccuts[1]) & (racuts[0] < Gleam_ra[i] < racuts[1]) & (Gleam_flux[i] > fluxmin))
 
    # #sources that fit our criteria
    data = table[w]
    Gleam_ra = data['RAJ2000']
    Gleam_dec = data['DEJ2000']
    Gleam_flux = data['Fintwide']
    
    
    # #open an instance of ds9
    ds9 = pyds9.DS9()
    
    for i in range(len(files)):
        print(files[i] + ' .... done')
        #opens wcs solutions in each header
        hdu = fits.open(files[i],mode='readonly', ignore_missing_end=True)
        imwcs = wcs.WCS(hdu[0].header)
        
        
        #loads each file into ds9 and circles GLEAM SOURCES
        hdu = fits.open(files[i],mode='readonly', ignore_missing_end=True)
        ds9.set_pyfits(hdu)
        ds9.set('frame {}'.format(i+1))
        ds9.set('file {}'.format(files[i]))
        ds9.set('scale zscale')
        ds9.set('zoom to fit')
        
        # Applies wcs solutions to translate GLEAM positions to pixel values of fits image
        refx,refy = imwcs.wcs_world2pix(Gleam_ra[:], Gleam_dec[:],0)  
        for i in range(len(Gleam_dec)):
            ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        
    return print('Success! Matched a total of {} sources!'.format(len(refx))) 

    

In [3]:
def pipeline(files,cat):
    Vizier.ROW_LIMIT = -1
    table = Vizier.get_catalogs(catalog=cat)[0]
    Gleam_ra = table['RAJ2000']
    Gleam_dec = table['DEJ2000']
    Gleam_flux = table['Fintwide']
    
    
    # #open an instance of ds9
    ds9 = pyds9.DS9()
    
    # for i in range(len(files)):
    print(files + ' .... done')
    #opens wcs solutions in each header
    hdu = fits.open(files[i],mode='readonly', ignore_missing_end=True)
    imwcs = wcs.WCS(hdu[0].header)


    #loads each file into ds9 and circles GLEAM SOURCES
    hdu = fits.open(files[i],mode='readonly', ignore_missing_end=True)
    ds9.set_pyfits(hdu)
    ds9.set('frame {}'.format(1))
    #ds9.set('file {}'.format(files))
    ds9.set('scale zscale')
    ds9.set('zoom to fit')

    # Applies wcs solutions to translate GLEAM positions to pixel values of fits image
    refx,refy = imwcs.wcs_world2pix(Gleam_ra[:], Gleam_dec[:],0)  
    for i in range(len(Gleam_dec)):
        ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))

    return print('Success! Matched a total of {} sources!'.format(len(refx))) 

    

In [4]:
pipeline('/home/gonzalo/astronomy_practice/Lynx_E_3.fits.gz','B/HST')

KeyboardInterrupt: 

In [3]:
deccuts , racuts, fluxmin = [-41,-21] , [0,161] , 3 #deg,deg,Jy
data = pd.read_csv('/home/gonzalo/Documents/LENC.csv')
ra = data['RA']
dec = data['DEC']
ds9 = pyds9.DS9()
#opens wcs solutions in each header


stokesI_hdu = fits.open('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorInewd.fits')
stokesQ_hdu = fits.open('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorQnewd.fits')
stokesU_hdu = fits.open('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorUnewd.fits')
stokesV_hdu = fits.open('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorVnewd.fits')
imwcs = wcs.WCS(stokesI_hdu[0].header)
refx,refy = imwcs.wcs_world2pix(ra, dec,0)  


ds9.set_pyfits(stokesI_hdu)
ds9.set('frame {}'.format(1))
ds9.set('file {}'.format('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorInewd.fits'))
ds9.set('scale zscale')
ds9.set('zoom to fit')
for i in range(len(ra)):
    ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))

ds9.set_pyfits(stokesQ_hdu)   
ds9.set('frame {}'.format(2))
ds9.set('file {}'.format('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorQnewd.fits'))
ds9.set('scale zscale')
ds9.set('zoom to fit')
for i in range(len(ra)):
    ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))

ds9.set_pyfits(stokesU_hdu)    
ds9.set('frame {}'.format(3))
ds9.set('file {}'.format('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorUnewd.fits'))
ds9.set('scale zscale')
ds9.set('zoom to fit')
for i in range(len(ra)):
    ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))

ds9.set_pyfits(stokesV_hdu)
ds9.set('frame {}'.format(4))
ds9.set('file {}'.format('/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorVnewd.fits'))
ds9.set('scale zscale')
ds9.set('zoom to fit')
for i in range(len(ra)):
    ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
# Applies wcs solutions to translate GLEAM positions to pixel values of fits image



In [4]:
#%%time
#here I specify the mosaiced fits files i want to use for the "pipeline"
#files = sorted(glob('masterpmaps/*.fits'))
#files = glob('/home/gonzalo/Desktop/polarizedmastermaps/*newd.fits')
#pipeline(files)

In [5]:
#what if i caclculate the flux of each gleam source using photo.utils
#and look to see which sources appear polarised but may not appear in
#different types of polarisations

In [6]:
#Assuming the faintest energy burst is about 10^31 erg and lasts for 
#10 minutes
energy_burst = 10e31 #in erg
time = 600 #in seconds

#1e7 erg / second conversion to watt
Luminosity = (energy_burst / time) * 1e-7

Frequency = 150e6 #150 Mhz
distance = 4.629e17 #meters in 15 pc

#this is in W/hz/m^2
flux = Luminosity / (4 * np.pi * Frequency * distance**2)

#to janksy conversion, 1e-23 W/hz/m^2 for 1 Jy
mjanksy_flux = (flux / 1e-23) * 1e3